<div style="background:#00000">
<div>
<h1>🧰 Starting Kit: <b>AUTO-SURVEY CHALLENGE'23</b> </h1>
<p>
This starting kit will guide you step by step and will walk you through the data statistics and examples. This will give you a clear idea of what this challenge is about and how you can proceed further to solve the challenge.
</p>
</div>
<div>
<br/><br/>
<details>
<summary open>Prerequisite</summary>
<p>
To run this notebook that calls the ChatGPT API you must get an <a href="https://platform.openai.com/docs/introduction">OpenAI API</a> by following <a href="https://github.com/kentrachmat/public_ai_paper_challenge_codabench/blob/master/how_to_get_openai_api.md">this instruction</a>.

</p>
</details>
<details>
<summary>Technical Details</summary>
<p>`
This code was tested with <a href="https://www.python.org/downloads/release/python-385/">Python 3.8.5</a> | <a href="https://anaconda.org/">Anaconda</a> custom (64-bit) | (default, Dec 23 2020, 21:19:02).
</p>
</details>

<details>
<summary>Disclaimer</summary>
  <p>
  ALL INFORMATION, SOFTWARE, DOCUMENTATION, AND DATA ARE PROVIDED "AS-IS". The CHALEARN, AND/OR OTHER ORGANIZERS OR CODE AUTHORS DISCLAIM ANY EXPRESSED OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR ANY PARTICULAR PURPOSE, AND THE WARRANTY OF NON-INFRIGEMENT OF ANY THIRD PARTY'S INTELLECTUAL PROPERTY RIGHTS. IN NO EVENT SHALL AUTHORS AND ORGANIZERS BE LIABLE FOR ANY SPECIAL,
  INDIRECT OR CONSEQUENTIAL DAMAGES OR ANY DAMAGES WHATSOEVER ARISING OUT OF OR IN CONNECTION WITH THE USE OR PERFORMANCE OF SOFTWARE, DOCUMENTS, MATERIALS, PUBLICATIONS, OR INFORMATION MADE AVAILABLE FOR THE CHALLENGE.
  </p>
</details>
<details>
<summary><a href="https://sites.google.com/view/ai-agents/credits?authuser=0">References and Credits </summary>
</details>
</div>
<hr>

<button type="button" ><h3><a href="https://www.codabench.org/competitions/866/">Competition Site</a></h3></button>
<button type="button" ><h3><a href="https://sites.google.com/view/ai-agents/home">Landing Page Site</a></h3></button>
<button type="button"><h3><a href="mailto:ai-agent-journal@chalearn.org">Contact us</a></h3></button>
<hr>
</div>

# Step 0: Input your API key

In [ ]:
import getpass

# Please enter your API key
new_api_key = getpass.getpass("Please input your API key: ")

# Step 1: Import required packages

In [ ]:
!git clone -q https://github.com/kentrachmat/public_ai_paper_challenge_codabench.git
%cd public_ai_paper_challenge_codabench
!pip install -q openai

In [3]:
import seaborn as sns; sns.set()
import warnings
import numpy as np
import json


warnings.simplefilter(action='ignore', category=FutureWarning)
%reload_ext autoreload

In [4]:
# DON'T CHANGE

MODEL_DIR = 'sample_code_submission/'
RESULT_DIR = 'sample_result_submission/'
PROBLEM_GENERATOR_DIR = 'generator_ingestion_program/'
SCORE_GENERATOR_DIR = 'generator_scoring_program/'
PROBLEM_REVIEWER_DIR = 'reviewer_ingestion_program/'
SCORE_REVIEWER_DIR = 'reviewer_scoring_program/'
DATA_DIR = 'sample_data'
DATA_NAME = 'ai_paper_challenge'
SCORING_OUTPUT_DIR = 'scoring_output'

file_paths = ['sample_code_submission/sample_submission_chatgpt_api_key.json',
              'generator_scoring_program/scoring_program_chatgpt_api_key.json',
              'reviewer_scoring_program/scoring_program_chatgpt_api_key.json']

for file_path in file_paths:
    with open(file_path, 'r') as file:
        json_data = json.load(file)

    json_data['key'] = new_api_key

    with open(file_path, 'w') as file:
        json.dump(json_data, file, indent=4)

***
# Step 2: Exploratory data analysis

We don't provide any TRAINING data, you are free to train on any external data. We only provide TEST data to evaluate your model.

`sample_data` contains:
- 3 prompts for the AI-Author track
- 3 set of papers for the AI-Reviewer track, each was generated by a single prompt.

### 🤖 `sample_data/generator`

This directory contains the necessary data for the AI-Author track

- `instructions.txt`: A text file that provides instructions or guidelines for generating a paper.
- `prompts.csv`: Automatically generated prompts by reverse-engineering original papers. These prompts are the input to your model.
- `metadata.csv`: Metadata from [Semantic Scholar](https://www.semanticscholar.org/) associates with the automatically generated prompts.
- `papers`: A subdirectory that contains variations of papers for each prompts.
  - 0, .. , N: Each number corresponds to a set of papers generated by a single prompt.

### 🖊️ `sample_data/reviewer`

This directory contains the necessary data for the AI-Reviewer track

- `instructions.txt`: A text file that provides instructions or guidelines for reviewing a paper.
- `papers`: A subdirectory that contains variations of paper that needs to be reviewed.
- `metadata.csv`: Informations each papers in the `papers` directory.

### Load Data

In [ ]:
from generator_ingestion_program.data_io import read_data as read_data_generator
from reviewer_ingestion_program.data_io import read_data as read_data_reviewer

data_generator, _ = read_data_generator(DATA_DIR, random_state=42)
data_reviewer, _ = read_data_reviewer(DATA_DIR, random_state=42)

### Data Statistics

In [ ]:
print("Total Generation prompts: ", len(np.unique(data_generator['generator']['ids'])))
print("Total Reviewer papers : ", len(np.unique(data_reviewer['reviewer']['ids'])))

### Visualization

#### AI-Author

 Instruction for a good survey paper

In [ ]:
print(data_generator['generator']['instructions'])

Prompts

In [ ]:
for p in data_generator['generator']['prompts']:
    print(p)
    print("Length:",len(p))
    print()

#### AI-Reviewer

Format of a review

In [ ]:
print(data_reviewer['reviewer']['instructions'])

Sample paper to be reviewed (bad language)

In [ ]:
sample_paper = data_reviewer['reviewer']['papers'][0]
for i, section in enumerate(sample_paper):
    if i in [0, 1, len(sample_paper)-1]:
        print(section)
    elif i==2:
      print('...')

***
# Step 3: Building a predictive model



In this competition, you need to implement a `model` class, with 2 functions:
- `generate_papers(prompts, instruction)`:
  - Arguments:
    - `prompts`: a list of strings represent given prompts
    - `instruction`: a string represents the instruction to generate good survey papers
  - Returns:
    - `generated_papers`: a list of strings in JSON format. The format of the string is describe in **Paper format**
- `review_papers(papers, instruction)`:
  - Arguments:
    - `papers`: a list of dictionaries represent papers to be reviewed. The format of each paper's dictionary is describe in **Paper format**.
    - `instruction`: a string represents the instruction to review papers.
  - Returns:
    - `reviews`: a list of dictionaries represent reviews. The format of each paper's review is describe in **Review format**.



## Paper format:

All papers in the competition follow this JSON format:
```python
[   
    {"heading":"Title", "text":"..."},
    {"heading":"Abstract", "text":"..."},
    .....
    .....
    {"heading":"Conclusion", "text":"..."},
    {"heading":"References", "text":"..."}
]
```

## Review format:
All reviews in the competition follow this JSON format:

```python
{
    "Responsibility": {
        "score": float,
        "comment": "comment A"
    },
    "Soundness": {
        "score": float,
        "comment": "comment B"
    },
    "Clarity":{
        "score": float,
        "comment": "comment C"
    },
    "Contribution": {
        "score": float,
        "comment": "comment D"
    },
    "Overall": {
        "score": float,
        "comment": "comment E"
    },
    "Confidence": {
        "score": float,
        "comment": "comment F"
    },
}
```
It is strongly recommended that you follow the exact naming and structure above.

## Baseline model
We have provided 2 baseline models:
- "Dummy" baseline:
  - `generate_papers()` returns the same template paper
  - `review_papers()` returns 0 score with no comment for every criterion

You can access this baseline in the `dummy_code_submission` folder
- ChatGPT baseline:
  - `generate_papers()` returns generated papers by ChatGPT, using the given prompts and instruction as inputs.
  - `review_papers()` returns generated reviews by ChatGPT, using the given papers and instruction as inputs.

You can access this baseline in the `sample_code_submission` folder.




In [ ]:
baseline = 'chatgpt' #@param ["chatgpt", "dummy"]


from generator_ingestion_program.data_io import write as write_generator
from reviewer_ingestion_program.data_io import write as write_reviewer

if baseline == "dummy":
  from dummy_code_submission.model import model
elif baseline == "chatgpt":
  from sample_code_submission.model import model
else:
  raise NotImplementedError

myModel = model()
generator_X = data_generator["generator"]["prompts"]
reviewer_X = data_reviewer["reviewer"]["papers"]

In [ ]:
# Generator
generator_Y_hat = myModel.generate_papers(generator_X, data_generator["generator"]["instructions"])

In [ ]:
# Reviewer
reviewer_Y_hat = myModel.review_papers(reviewer_X, data_reviewer["reviewer"]["instructions"])

In [ ]:
result_name = RESULT_DIR + DATA_NAME

# Output predicted papers
write_generator(result_name + '_generator.predict', generator_Y_hat)

# Output reviews
write_reviewer(result_name + '_reviewer.predict', reviewer_Y_hat)

# Check if they exists
!ls $result_name*

***
# Step 4: Evaluation


### Scoring program
You can evaluate your model locally using this sample scoring program (using ChatGPT only). **Our Scoring Program** with sophisticated analysis is **hidden** until the competition ends.

### Scoring results
- If you run locally, you can find all results in the `scoring_output/` folder.

- If you submit your solution to Codabench, you can download the results by choosing "Output from scoring step". The evaluation on Codabench is implemented using **Our Scoring Program**.

## AI-Author evaluation

In [ ]:
!python3 $SCORE_GENERATOR_DIR/score.py $DATA_DIR $RESULT_DIR $SCORING_OUTPUT_DIR

## AI-Reviewer evaluation

In [ ]:
!python3 $SCORE_REVIEWER_DIR/score.py $DATA_DIR $RESULT_DIR $SCORING_OUTPUT_DIR

***
# Step 5: Submission

The code below create a zip file that you can submit to Codabench.

In [ ]:
import datetime
from generator_ingestion_program.data_io import zipdir

the_date = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
sample_code_submission = 'sample_code_submission_' + the_date + '.zip'
zipdir(sample_code_submission, MODEL_DIR, exclude_folders=['__pycache__'], exclude_files=[f'{DATA_NAME}_model.pickle'])
print("Submit this file to codalab:\n" + sample_code_submission)